# Setup

In [ ]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
import pyarrow as pa
import fastparquet as fp
import seaborn as sns
import matplotlib.pyplot as plt
import os

# for jupyter notebook, this is necessary to show plots
%matplotlib inline

In [ ]:
# # this does not work as it is admin (root)
# default_dir = '/bd-fs-mnt/'
# print(os.getcwd())

In [ ]:
# change current working directory
default_dir = '..'

if os.getcwd() != default_dir:
    os.chdir(default_dir)

In [ ]:
print(os.getcwd())

In [ ]:
# initalize input directory
input_dir = '..'

# read parquet files into dataframes
df_vitals = pd.read_parquet(input_dir + '/Vitals/')
df_surgery = pd.read_parquet(input_dir + '/Surgery/')
df_reanimatie = pd.read_parquet(input_dir + '/Reanimatiebeleid/')
df_lab = pd.read_parquet(input_dir + '/Lab/')
df_ic = pd.read_parquet(input_dir + '/IC_Opnames/')
df_demo = pd.read_parquet(input_dir + '/Demographics/')

# list of dataframes
dataframes = [df_vitals, df_surgery, df_reanimatie, df_lab, df_ic, df_demo]

In [ ]:
# print memory usage
for df in dataframes:
    # df.memory_usage() does not consider the size of objects when deep=False
    memory = df.memory_usage(deep=True).sum() / (1024 ** 2)
print(f"Total memory usage: {memory} MB")

# EDA using ydata profiling and extra code

We will explore each data in this section.    
We first generate automated reports, verify some questions based on the report, and then execute extra codes

In [ ]:
# initialize titles 
titles = ["vitals", "surgery", "reanimatie", "lab", "ic_opnames", "demographics"]

# specify the output directory for saving html files
output_dir = '..'

# create the folder if not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# save report for each dataframe
for df, title in zip(dataframes, titles):
    output_path = os.path.join(output_dir, f"{title}.html")

    # check if the file already exists
    if not os.path.isfile(output_path):
        try:
            profile = ProfileReport(df, title=title, minimal=True)
            profile.to_file(output_path)
            
        # skips if the data gives an error
        except Exception as e:
            print(f"This data gives an error for {title}: {e}. Try EDA manually.")
            continue

## Vitals

### Verification for interesting parts based on the report

Q: Check whether the percentage of distinct patients, and opname is indeed low   
A: Indeed low and matches to the report

In [ ]:
print(f"Numbers of distinct values: {df_vitals['PID'].nunique()}")
print(f"Numbers of total values: {df_vitals['PID'].count()} \n")

print(f"Numbers of distinct values: {df_vitals['OpnameID'].nunique()}")
print(f"Numbers of total values: {df_vitals['OpnameID'].count()}")

Q: For each distinct PID, how many there would be?

In [ ]:
print(df_vitals['PID'].value_counts().head(10))
df_vitals[df_vitals['PID']=='..'].head(5)

Q: Check whether there are many measurements related to heart rates   
A: Indeed

In [ ]:
print(str(df_vitals.MetingOms.value_counts()) + "\n")
print(str(df_vitals.MetingEenheid.value_counts()) + "\n")

MetingDatumTijd   
Minimum: 2018-12-01 02:03:31   
Maximum: 2023-02-01 20:24:00   

In [ ]:
print(df_vitals['MetingDatumTijd'].sort_values())

Q: Check missing values per column. 

In [ ]:
print(df_vitals.isna().sum())

Q: Show rows that have missing values

In [ ]:
df_vitals[df_vitals.isnull().any(axis=1)].head(5)

Q: Why there's a negative value? NIBP being negative is possible?  
A: 

In [ ]:
pd.set_option('display.float_format', '{:.1f}'.format)
print(df_vitals['MeetWaarde3'].value_counts().sort_index())
df_vitals[df_vitals['MeetWaarde3'] == -1.0]

Q: Why is maximum super high? NIBP being this high is possible?   
A: 

In [ ]:
df_vitals[df_vitals['MeetWaarde3'] == 2147482624.0]

Q: Why many 'MeetWaarde's missing? (When do you measure only once, twice, three times? Does it makes sense for all Meting? How should I handle this missing values?     
A: 

In [ ]:
df_vitals[df_vitals['MeetWaarde1'].isna()]

In [ ]:
df_vitals[df_vitals['MeetWaarde3'].isna()]

Q: What's the zeros? do we actually have 0's? count 914266?   
A: Yes

In [ ]:
print((df_vitals['MeetWaarde1'] == 0).sum())
df_vitals[df_vitals['MeetWaarde1'] == 0]

Q: Does it make sense that each Meting can be 0? If so/ If not, how should I handle this?   
A: 

In [ ]:
filtered = df_vitals[df_vitals['MeetWaarde1'] == 0]
filtered['Meting'].value_counts()

Q: Should we ignore the MeetOptie column? If not, what would they mean?   
A:

In [ ]:
df_vitals['MeetOptie'].unique().tolist()[:5]

### Extra

In [ ]:
print(df_vitals.head(5))
print("\n\n")
print(df_vitals.Meting.unique())
print("\n\n")
print(df_vitals.info())
print("\n\n")
print(df_vitals.describe())
print("\n\n")
print('Data`s Shape: ', df_vitals.shape)
print("\n\n")
print('\nType of features \n', df_vitals.dtypes.value_counts())
print("\n\n")
isna_series = df_vitals.isna().sum()
print('\nNull columns and numbers:\n ', isna_series[isna_series > 0].sort_values(ascending=False))

Boxplot for numerical variables

In [ ]:
# TODO: dropped missing values for now. but may need correction later 
# Also this is just purely based on the values, each Meting type is not taken into account.
numerical_vitals = df_vitals.select_dtypes(include=['number']).dropna()

for col in numerical_vitals.columns:
    plt.figure()
    numerical_vitals.boxplot(column=col)
    plt.title(f"Boxplot of {col}")
    plt.ylabel('Values')
    plt.show()

This is wrong conclusion. Ignore this.   
Correlation and heatmap.   
Conclusion: MeetWaarde1 (MW1) and MW2 show a moderately strong linear relationship (increase / decrease)   
MW1 and MW3 do not show meaningful relationship.    
Same for MW2 and MW3 

In [ ]:
# # calculate the correlation matrix
# # this is wrong correlation matrix, as some 'Meting' type only has MeetWaarde1
# corr_matrix_vitals = numerical_vitals.corr()

# # show a heatmap
# plt.figure(figsize=(10, 8))
# sns.heatmap(corr_matrix_vitals, annot=True)
# plt.title("Correlation of Vitals")
# plt.show()

MeetWaarde values per each Meting   
Q: Should I carry more tests like this (e.g. MeetWaarde / something) ?

In [ ]:
MW1_per_meting = df_vitals.groupby('Meting')['MeetWaarde1'].agg(['mean', 'median', 'std', 'count'])
MW1_per_meting

In [ ]:
MW2_per_meting = df_vitals.groupby('Meting')['MeetWaarde2'].agg(['mean', 'median', 'std', 'count'])
MW2_per_meting

In [ ]:
MW3_per_meting = df_vitals.groupby('Meting')['MeetWaarde3'].agg(['mean', 'median', 'std', 'count'])
MW3_per_meting

# check missing values
# m = df_vitals[df_vitals['MeetWaarde3'].notna()]
# m['Meting'].value_counts()

### Per column, meaning of values (and how to handle missing values?)

Q: Meaning? 

In [ ]:
df_vitals['Meting'].value_counts()

Q: Hartritme only 3? Same as Heart Rate?

In [ ]:
df_vitals['MetingOms'].value_counts()

In [ ]:
df_vitals.loc[df_vitals['MetingOms'] == 'Hartritme']

Q: Empty metingeenheid? 1/min, bpm?

In [ ]:
unique = df_vitals.groupby(['Meting', 'MetingEenheid', 'MetingOms']).size().reset_index(name='Counts')
unique

## Surgery

### Verification for interesting parts based on the report

Q: Is OperatieID each surgery ID? Because they're all distinct   
A:   

Q: What was OK_BeginDatumTijd again?   
A:   

Q: Why some missing from OK_EindDatumTijd?   
A:   

Q: Should we ignore NaarAfedlingDatum? or OK_EindDatumTijd? Also why some missing in both?? Why distinct numbers are different between them?       
A:   

Q: HoofdverrichtingCode. Types of surgery/department. What does the number mean? Important? Why only number '332100'?   
A:   

Q: HoofdverrichtingOms important?   
A:   

Q: PrioriteitCode what does this mean? Do we want to include them, for what? I guess we can use PrioriteitCode instead of Oms.   
A:   

Q: PrioriteitOms meaning ?Do we want to include them?    
A:   

In [ ]:
df_surgery.loc[df_surgery['OK_EindDatumTijd'].isna()]

In [ ]:
df_surgery['HoofdverrichtingCode'].value_counts()

In [ ]:
df_surgery['HoofdverrichtingOms'].value_counts()

In [ ]:
df_surgery['PrioriteitCode'].value_counts()

In [ ]:
df_surgery['PrioriteitOms'].value_counts()

### Extra

In [ ]:
print(df_surgery.head(5))
print("\n\n")
print(df_surgery.HoofdverrichtingOms.unique())
print("\n\n")
print(df_surgery.info())
print("\n\n")
print(df_surgery.describe())
print("\n\n")
print('Data`s Shape: ', df_surgery.shape)
print("\n\n")
print('\nType of features \n', df_surgery.dtypes.value_counts())
print("\n\n")
isna_series = df_surgery.isna().sum()
print('\nNull columns and numbers:\n ', isna_series[isna_series > 0].sort_values(ascending=False))

### Per column, meaning of values (and how to handle missing values?)

Q: Meaning?

In [ ]:
# show different codes with different length of the first item
unique_codes = df_surgery['HoofdverrichtingCode'].unique().tolist()
diff_codes = [code for code in unique_codes if len(code) != len(unique_codes[0])]
print(diff_codes or "All items have equal length.")

In [ ]:
unique = df_surgery.groupby(['HoofdverrichtingCode', 'HoofdverrichtingOms']).size().reset_index(name='Counts')[:5]
unique

Q: What does this mean?

In [ ]:
unique = df_surgery.groupby(['PrioriteitCode', 'PrioriteitOms']).size().reset_index(name='Counts')
unique

## Reanimatiebeleid

### Verification for interesting parts based on the report

Q: Opname ID isn't fully distinct   
A: Several attempts for a same patient on different dates. But is PID unique on its own (regardless of distinct)?  

Q: What's Gekozen_beleid?   
A:   

Q: Why so many missing in 'Reanimeren'? And what's 'niet beslist'?   
A:   

Q: What's 'Care_order'?   
A:   

Q: What's VanafDatum?   
A:   

Q: What's TmDatum?   
A:   

Q: Why TmDatum minimum looks weird? Minimum    1816-03-30 05:55:08.066277    
A:   

In [ ]:
# mark all duplicated OpnameID
df_reanimatie[df_reanimatie['OpnameID'].duplicated(keep=False)]

In [ ]:
df_reanimatie['Gekozen_beleid'].value_counts()

In [ ]:
df_reanimatie.loc[df_reanimatie['Reanimeren'].isna()]

In [ ]:
df_reanimatie['TmDatum'].unique().tolist()[:5]

### Extra

In [ ]:
print(df_reanimatie.head(5))
print("\n\n")
print(df_reanimatie.VanafDatum.unique())
print("\n\n")
print(df_reanimatie.info())
print("\n\n")
print(df_reanimatie.describe())
print("\n\n")
print('Data`s Shape: ', df_reanimatie.shape)
print("\n\n")
print('\nType of features \n', df_reanimatie.dtypes.value_counts())
print("\n\n")
isna_series = df_reanimatie.isna().sum()
print('\nNull columns and numbers:\n ', isna_series[isna_series > 0].sort_values(ascending=False))

### Per column, meaning of values (and how to handle missing values?)

Q: meaning?

In [ ]:
unique = df_reanimatie.groupby(['Gekozen_beleid', 'Reanimeren', 'Care_order']).size().reset_index(name='Counts')
unique

## Lab

### Verification for interesting parts based on the report

Q: BepalingCode what's this   
Q: GroepOms what's this distinction?   
Q: GlimsCode what's this   
Q: Uitslag what's this. It's not just number.   
Q: Labdatumtijd, is it when they started testing things?   

In [ ]:
df_lab['BepalingCode'].value_counts()

In [ ]:
df_lab['GroepOms'].value_counts()

In [ ]:
df_lab['GlimsCode'].value_counts()

In [ ]:
df_lab['Uitslag'].value_counts()

In [ ]:
df_lab['Labdatumtijd'].value_counts()

### Extra

In [ ]:
print(df_lab.head(5))
print("\n\n")
print(df_lab.GlimsCode.unique())
print("\n\n")
print(df_lab.info())
print("\n\n")
print(df_lab.describe())
print("\n\n")
print('Data`s Shape: ', df_lab.shape)
print("\n\n")
print('\nType of features \n', df_lab.dtypes.value_counts())
print("\n\n")
isna_series = df_lab.isna().sum()
print('\nNull columns and numbers:\n ', isna_series[isna_series > 0].sort_values(ascending=False))

### Per column, meaning of values (and how to handle missing values?)

Q: meaning?

In [ ]:
unique = df_lab.groupby(['BepalingCode', 'BepalingOms', 'GroepOms', 'GlimsCode']).size().reset_index(name='Counts')
unique

Q: Empty eendheid. What should I do? 

In [ ]:
df_lab['Eenheid'].value_counts()

In [ ]:
df_lab['Uitslag'].value_counts()[:5]

In [ ]:
unique = df_lab.groupby(['Eenheid', 'Uitslag']).size().reset_index(name='Counts')[:100]
unique

## IC_Opnames

Q: Is opnameID here different from other dataframes, as it is IC and others are general wards?    
Q: Ontslag to general ward? Or home?   
Q: SpecialismeCode. What's this?   
Q: AfdelingCode?   

In [ ]:
df_ic['SpecialismeCode'].value_counts()

In [ ]:
df_ic['AfdelingCode'].value_counts()

### Extra

In [ ]:
print(df_ic.head(5))
print("\n\n")
print(df_ic.SpecialismeCode.unique())
print("\n\n")
print(df_ic.info())
print("\n\n")
print(df_ic.describe())
print("\n\n")
print('Data`s Shape: ', df_ic.shape)
print("\n\n")
print('\nType of features \n', df_ic.dtypes.value_counts())
print("\n\n")
isna_series = df_ic.isna().sum()
print('\nNull columns and numbers:\n ', isna_series[isna_series > 0].sort_values(ascending=False))

### Per column, meaning of values (and how to handle missing values?)

Q: Meaning?

In [ ]:
unique = df_ic.groupby(['SpecialismeCode', 'AfdelingCode']).size().reset_index(name='Counts')
unique

## Demographics

### Verification for interesting parts based on the report

Q: OverlijdensDatum missing means still alive? Or don't know?     

Q: Is OpnameDatumTijd the same as the one from other dataframe? (e.g. ic_opnames)   
A: Ofc not. Because IC_opnames is only for people who've been to IC. Demographics include general wards.   

Q: Does OntslagDatumTijd missing means death? Why minimum is weird. Why there's a missing?   
Q: What's SpecialismeCode?   
Q: Spoed (n: nee, j: ja)   
Q: What's OpnameTypeOms?   

In [ ]:
df_demo.loc[df_demo['OverlijdensDatum'].isna()][:5]

In [ ]:
df_demo['OpnameDatumTijd'].nunique()

In [ ]:
df_ic['OpnameDatumTijd'].nunique()

In [ ]:
df_demo.loc[df_demo['OntslagDatumTijd'].isna()]

In [ ]:
df_demo['OntslagDatumTijd'].min()

In [ ]:
min_date = df_demo['OntslagDatumTijd'].min()
df_demo.loc[df_demo['OntslagDatumTijd'] == min_date]

In [ ]:
df_demo['SpecialismeCode'].value_counts()

In [ ]:
df_demo['OpnameTypeOms'].value_counts()

### Extra

In [ ]:
print(df_demo.head(5))
print("\n\n")
print(df_demo.SpecialismeCode.unique())
print("\n\n")
print(df_demo.info())
print("\n\n")
print(df_demo.describe())
print("\n\n")
print('Data`s Shape: ', df_demo.shape)
print("\n\n")
print('\nType of features \n', df_demo.dtypes.value_counts())
print("\n\n")
isna_series = df_ic.isna().sum()
print('\nNull columns and numbers:\n ', isna_series[isna_series > 0].sort_values(ascending=False))

In [ ]:
leeftijd_per_specialismecode = df_demo.groupby('SpecialismeCode')['Leeftijd'].agg(['mean', 'median', 'std', 'count'])
leeftijd_per_specialismecode

### Per column, meaning of values (and how to handle missing values?)

Q: Meaning?   

Just a thought: There are more women in early ages due to birth maybe? 

In [ ]:
unique = df_demo.groupby(['Geslacht', 'Leeftijd']).size().reset_index(name='Counts')
plt.figure(figsize=(18, 6))
sns.barplot(data=unique, x='Leeftijd', y='Counts', hue='Geslacht')
plt.title('Counts of Unique Combis of Geslacht and Leeftijd')
plt.xlabel('Leeftijd')
plt.ylabel('Counts')
plt.legend(title='Counts')
plt.tight_layout()
plt.show()

In [ ]:
unique = df_demo.groupby(['SpecialismeCode', 'Spoed', 'OpnameTypeOms']).size().reset_index(name='Counts')
unique

In [ ]:
# check patients who passed away based on age
df_demo_f = df_demo[df_demo['OverlijdensDatum'].notna()]
unique = df_demo_f.groupby(['Leeftijd']).size().reset_index(name='Counts')
unique

In [ ]:
# check patients who passed away based on age and sex 
df_demo_f = df_demo[df_demo['OverlijdensDatum'].notna()]
unique = df_demo_f.groupby(['Geslacht', 'Leeftijd']).size().reset_index(name='Counts')

plt.figure(figsize=(18, 6))
sns.barplot(data=unique, x='Leeftijd', y='Counts', hue='Geslacht')
plt.title('Counts of Unique Combis of Dead Patients Geslacht and Leeftijd')
plt.xlabel('Leeftijd')
plt.ylabel('Counts')
plt.legend(title='Geslacht')
plt.tight_layout()
plt.show()

In [ ]:
# check ages 
counts = df_demo['Leeftijd'].value_counts()
counts.columns = ['Leeftijd', 'Counts']

plt.figure(figsize=(18, 6))
sns.barplot(data=unique, x='Leeftijd', y='Counts')
plt.title('Counts of Ages')
plt.xlabel('Leeftijd')
plt.ylabel('Counts')
plt.tight_layout()
plt.show()